In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import numpy as np
import scipy.stats as st


In [1]:
#import data files
nyc_crimedata_path = "clean_data/NYC_full_21_22.csv"
nyc_poppath_path = "clean_data/nyc_pop_data.csv"



In [4]:
#Read nyc crime data 
nyc_crime_df = pd.read_csv(nyc_crimedata_path)
nyc_crime_df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,261170525,11/01/2022,15:00:00,11/01/2022,15:15:00,62.0,12/31/2022,578,HARRASSMENT 2,638.0,...,PATROL BORO BKLYN SOUTH,(null),25-44,WHITE,F,13511.0,1.0,2.0,44.0,37.0
1,261162294,12/03/2022,01:20:00,12/03/2022,04:00:00,68.0,12/31/2022,341,PETIT LARCENY,321.0,...,PATROL BORO BKLYN SOUTH,(null),25-44,WHITE,M,13511.0,10.0,2.0,44.0,41.0
2,261176844,11/14/2022,18:23:00,11/14/2022,18:30:00,90.0,12/31/2022,578,HARRASSMENT 2,637.0,...,PATROL BORO BKLYN NORTH,(null),45-64,BLACK,F,17213.0,36.0,2.0,30.0,56.0
3,261165498,11/28/2022,09:00:00,12/26/2022,09:00:00,84.0,12/31/2022,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,...,PATROL BORO BKLYN NORTH,(null),25-44,BLACK,F,16865.0,68.0,2.0,38.0,54.0
4,261167687,08/01/2022,09:00:00,12/31/2022,09:00:00,121.0,12/31/2022,341,PETIT LARCENY,349.0,...,PATROL BORO STATEN ISLAND,(null),25-44,WHITE,M,10371.0,4.0,1.0,13.0,75.0


In [5]:
#Keep relevant nyc crime data columns
clean_crime_data = nyc_crime_df[["LAW_CAT_CD", "BORO_NM", "SUSP_AGE_GROUP", "SUSP_RACE", "SUSP_SEX", "Latitude","Longitude","Lat_Lon", "VIC_AGE_GROUP" ,"VIC_RACE" ,"VIC_SEX" ]].copy()

clean_crime_data

,LAW_CAT_CD,BORO_NM,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,VIOLATION,BROOKLYN,45-64,WHITE,M,40.621708,-74.000593,"(40.621708, -74.000593)",25-44,WHITE,F
1,MISDEMEANOR,BROOKLYN,(null),(null),(null),40.618867,-74.021057,"(40.618867, -74.021057)",25-44,WHITE,M
2,VIOLATION,BROOKLYN,25-44,BLACK,M,40.705035,-73.939373,"(40.705035, -73.939373)",45-64,BLACK,F
3,MISDEMEANOR,BROOKLYN,25-44,BLACK,U,40.700294,-73.990838,"(40.700294, -73.990838)",25-44,BLACK,F
4,MISDEMEANOR,STATEN ISLAND,(null),(null),(null),40.629066,-74.166448,"(40.62906605, -74.16644829)",25-44,WHITE,M
...,...,...,...,...,...,...,...,...,...,...,...
520656,MISDEMEANOR,MANHATTAN,UNKNOWN,BLACK,F,40.820654,-73.939794,"(40.820654, -73.939794)",25-44,BLACK,F
520657,MISDEMEANOR,QUEENS,25-44,WHITE HISPANIC,M,40.751091,-73.860642,"(40.751091, -73.860642)",25-44,WHITE HISPANIC,M
520658,FELONY,QUEENS,25-44,WHITE HISPANIC,M,40.751091,-73.860642,"(40.751091, -73.860642)",25-44,WHITE HISPANIC,M
520659,FELONY,BROOKLYN,UNKNOWN,BLACK,M,40.623411,-73.929796,"(40.623411, -73.929796)",18-24,BLACK,M


In [14]:
#Read nyc pop data 
nyc_pop_df = pd.read_csv(nyc_poppath_path)
nyc_pop_df.head()

,borough,pop
0,bronx,1379946
1,brooklyn,2590516
2,manhattan,1596273
3,queens,2278029
4,staten island,491133


In [17]:
#Convert nyc pop data to uppercase and rename column header
nyc_pop_df['borough'] = nyc_pop_df['borough'].str.upper()
renamed_nyc_pop_df = nyc_pop_df.rename(columns={"borough": "BORO_NM"})

renamed_nyc_pop_df

,BORO_NM,pop
0,BRONX,1379946
1,BROOKLYN,2590516
2,MANHATTAN,1596273
3,QUEENS,2278029
4,STATEN ISLAND,491133


In [18]:
#Combine data into single dataframe 
combined_data = pd.merge(clean_crime_data, renamed_nyc_pop_df, on="BORO_NM")

combined_data.head()


,LAW_CAT_CD,BORO_NM,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,pop
0,VIOLATION,BROOKLYN,45-64,WHITE,M,40.621708,-74.000593,"(40.621708, -74.000593)",25-44,WHITE,F,2590516
1,MISDEMEANOR,BROOKLYN,(null),(null),(null),40.618867,-74.021057,"(40.618867, -74.021057)",25-44,WHITE,M,2590516
2,VIOLATION,BROOKLYN,25-44,BLACK,M,40.705035,-73.939373,"(40.705035, -73.939373)",45-64,BLACK,F,2590516
3,MISDEMEANOR,BROOKLYN,25-44,BLACK,U,40.700294,-73.990838,"(40.700294, -73.990838)",25-44,BLACK,F,2590516
4,MISDEMEANOR,BROOKLYN,45-64,WHITE,M,40.622093,-73.998140,"(40.622093, -73.99814)",UNKNOWN,UNKNOWN,D,2590516


In [25]:
#Rename col headers 

renamed_df = combined_data.rename(columns={"LAW_CAT_CD": "offence level", "BORO_NM": "borough",
                                            "SUSP_AGE_GROUP": "suspect age group", "SUSP_RACE": "suspect race", "SUSP_SEX": "suspect sex",
                                             "VIC_AGE_GROUP": "victim age group", "VIC_RACE": "victim race", "VIC_SEX": "victim sex"})

renamed_df.head()


,offence level,borough,suspect age group,suspect race,suspect sex,Latitude,Longitude,Lat_Lon,victim age group,victim race,victim sex,pop
0,VIOLATION,BROOKLYN,45-64,WHITE,M,40.621708,-74.000593,"(40.621708, -74.000593)",25-44,WHITE,F,2590516
1,MISDEMEANOR,BROOKLYN,(null),(null),(null),40.618867,-74.021057,"(40.618867, -74.021057)",25-44,WHITE,M,2590516
2,VIOLATION,BROOKLYN,25-44,BLACK,M,40.705035,-73.939373,"(40.705035, -73.939373)",45-64,BLACK,F,2590516
3,MISDEMEANOR,BROOKLYN,25-44,BLACK,U,40.700294,-73.990838,"(40.700294, -73.990838)",25-44,BLACK,F,2590516
4,MISDEMEANOR,BROOKLYN,45-64,WHITE,M,40.622093,-73.998140,"(40.622093, -73.99814)",UNKNOWN,UNKNOWN,D,2590516


In [34]:
#Check for incomplete rows
renamed_df.count()


offence level        519678
borough              519678
suspect age group    519678
suspect race         519678
suspect sex          519678
Latitude             519675
Longitude            519675
Lat_Lon              519675
victim age group     519678
victim race          519678
victim sex           519678
pop                  519678
dtype: int64

In [35]:
#Check colomuns for any data errors 
offence_level_check = renamed_df["offence level"].value_counts()
offence_level_check


offence level
MISDEMEANOR    263708
FELONY         172896
VIOLATION       83074
Name: count, dtype: int64

In [36]:
#Check colomuns for any data errors 
borough_check = renamed_df["borough"].value_counts()
borough_check


borough
BROOKLYN         144366
MANHATTAN        130972
QUEENS           112248
BRONX            110031
STATEN ISLAND     22061
Name: count, dtype: int64

In [45]:
#Check colomuns for any data errors 
suspect_age_group_check = renamed_df["suspect age group"].value_counts()
suspect_age_group_check

suspect age group
UNKNOWN    200571
25-44      133332
(null)      91055
45-64       45872
18-24       34339
<18          9474
65+          4998
2022           17
-6              2
-941            1
-10             1
956             1
952             1
-65             1
-3              1
-66             1
-47             1
-961            1
-951            1
924             1
-969            1
-55             1
-5962           1
1822            1
951             1
-76             1
-1              1
Name: count, dtype: int64

In [58]:
#Clean up suspect age group column (UNKNOWN)
suspect_age_list = ['<18', '18-24', '25-44', '45-64', '65+']
renamed_df.loc[~renamed_df['suspect age group'].isin(suspect_age_list),'suspect age group'] = "Unknown"

In [59]:
#Check suspect age group 
suspect_age_group_check = renamed_df["suspect age group"].value_counts()
suspect_age_group_check

suspect age group
Unknown    291663
25-44      133332
45-64       45872
18-24       34339
<18          9474
65+          4998
Name: count, dtype: int64

In [38]:
#Check colomuns for any data errors 
suspect_race_check = renamed_df["suspect race"].value_counts()
suspect_race_check

suspect race
BLACK                             153819
UNKNOWN                           130796
(null)                             91055
WHITE HISPANIC                     68193
WHITE                              37675
BLACK HISPANIC                     22077
ASIAN / PACIFIC ISLANDER           15125
AMERICAN INDIAN/ALASKAN NATIVE       938
Name: count, dtype: int64

In [64]:
#Clean up suspect race column (UNKNOWN)
suspect_race_list = ['BLACK', 'WHITE HISPANIC', 'WHITE', 'BLACK HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKAN NATIVE']
renamed_df.loc[~renamed_df['suspect race'].isin(suspect_race_list),'suspect race'] = "Unknown"
suspect_race_check = renamed_df["suspect race"].value_counts()
suspect_race_check

suspect race
Unknown                           221851
BLACK                             153819
WHITE HISPANIC                     68193
WHITE                              37675
BLACK HISPANIC                     22077
ASIAN / PACIFIC ISLANDER           15125
AMERICAN INDIAN/ALASKAN NATIVE       938
Name: count, dtype: int64

In [65]:
#Check colomuns for any data errors 
suspect_sex_check = renamed_df["suspect sex"].value_counts()
suspect_sex_check

suspect sex
M          251029
Unknown    201244
F           67405
Name: count, dtype: int64

In [63]:
#Clean up suspect sex column (UNKNOWN)
suspect_sex_list = ['M', 'F']
renamed_df.loc[~renamed_df['suspect sex'].isin(suspect_sex_list),'suspect sex'] = "Unknown"
suspect_sex_check = renamed_df["suspect sex"].value_counts()
suspect_sex_check

suspect sex
M          251029
Unknown    201244
F           67405
Name: count, dtype: int64

In [40]:
#Check colomuns for any data errors 
victim_age_check = renamed_df["victim age group"].value_counts()
victim_age_check

victim age group
25-44      185071
UNKNOWN    148308
45-64       98644
18-24       43802
65+         25740
<18         18089
-65             2
-5              2
-57             1
-49             1
-3              1
-964            1
-6              1
-963            1
-40             1
-960            1
-970            1
934             1
-955            1
-959            1
-945            1
-27             1
-11             1
-971            1
946             1
950             1
-975            1
-934            1
Name: count, dtype: int64

In [66]:
#Clean up victim age group column (UNKNOWN)
victim_age_list = ['<18', '18-24', '25-44', '45-64', '65+']
renamed_df.loc[~renamed_df['victim age group'].isin(victim_age_list),'victim age group'] = "Unknown"
victim_age_check = renamed_df["victim age group"].value_counts()
victim_age_check

victim age group
25-44      185071
Unknown    148332
45-64       98644
18-24       43802
65+         25740
<18         18089
Name: count, dtype: int64

In [41]:
#Check colomuns for any data errors 
victim_sex_check = renamed_df["victim sex"].value_counts()
victim_sex_check

victim sex
F    198607
M    180811
D     95164
E     43283
L      1813
Name: count, dtype: int64

In [67]:
#Clean up victim sex column (UNKNOWN)
victim_sex_list = ['M', 'F']
renamed_df.loc[~renamed_df['victim sex'].isin(victim_sex_list),'victim sex'] = "Unknown"
victim_sex_check = renamed_df["victim sex"].value_counts()
victim_sex_check

victim sex
F          198607
M          180811
Unknown    140260
Name: count, dtype: int64

In [68]:
#group by victim
age_victim = renamed_df.groupby(["victim age group"])["offence level"].value_counts()

age_victim


victim age group  offence level
18-24             MISDEMEANOR      20566
                  FELONY           15327
                  VIOLATION         7909
25-44             MISDEMEANOR      84301
                  FELONY           62405
                  VIOLATION        38365
45-64             MISDEMEANOR      43549
                  FELONY           31604
                  VIOLATION        23491
65+               FELONY           10715
                  MISDEMEANOR       9212
                  VIOLATION         5813
<18               MISDEMEANOR       9626
                  FELONY            4737
                  VIOLATION         3726
Unknown           MISDEMEANOR      96454
                  FELONY           48108
                  VIOLATION         3770
Name: count, dtype: int64

In [70]:
#export cleaned crime dataframe to CSV file 

renamed_df.to_csv("clean_data/cleaned_nyc_crime_data.csv", index=False, header=True)